In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.applications.mobilenet import MobileNet as cnn
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import layers
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, Input, MaxPool2D
from tensorflow.keras import Model
from keras.layers import Input, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
# len(os.listdir('/kaggle/input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA'))

In [ ]:
ls /kaggle/input/chest-xray-pneumonia/chest_xray/test

In [ ]:
nb_train_samples = len(os.listdir(train_dir + '/NORMAL')) + len(os.listdir(train_dir + '/PNEUMONIA'))
print(nb_train_samples)
nb_val_samples  = len(os.listdir(val_dir + '/NORMAL')) + len(os.listdir(val_dir + '/PNEUMONIA'))
print(nb_val_samples)
nb_test_samples  = len(os.listdir(test_dir + '/NORMAL')) + len(os.listdir(val_dir + '/PNEUMONIA'))
print(nb_test_samples)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                      shear_range=0.2,zoom_range=0.2,
                                      horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale=1./ 255)

batch_size = 10
img_width = 150
img_height = 150

train_generator = train_datagen.flow_from_directory(train_dir,batch_size = batch_size,
                                                    class_mode = 'binary', 
                                                    target_size =(img_width, img_height))

val_generator = val_datagen.flow_from_directory(val_dir,batch_size = batch_size,
                                                    class_mode = 'binary', 
                                                    target_size =(img_width, img_height))



test_generator = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,                                              
                                                class_mode='binary',
                                                target_size=(img_width, img_height))



x_train,y_train=train_generator.next()
x_test,y_test=test_generator.next(624)

from pathlib import Path
# Define path to the data directory
data_dir = Path('../input/chest-xray-pneumonia/chest_xray/chest_xray')

# Path to train directory (Fancy pathlib...no more os.path!!)
train_dir = data_dir / 'train'
import cv2
from keras.utils import to_categorical
# Path to validation directory
val_dir = data_dir / 'val'

# Path to test directory
test_dir = data_dir / 'test'

#Get the path to the sub-directories
normal_cases_dir = train_dir / 'NORMAL'
pneumonia_cases_dir = train_dir / 'PNEUMONIA'

# Get the list of all the images
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

# List that are going to contain validation images data and the corresponding labels
valid_data = []
valid_labels = []
import numpy as np

# Some images are in grayscale while majority of them contains 3 channels. So, if the image is grayscale, we will convert into a image with 3 channels.
# We will normalize the pixel values and resizing all the images to 224x224 

# Normal cases
for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (150,150))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150,150)).astype(np.float32)/255.0
    label = to_categorical(0, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
                      
# Pneumonia cases        
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (150,150))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.0
    label = to_categorical(1, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
    
# Convert the list into numpy arrays
x_train = np.array(valid_data)
y_train = np.array(valid_labels)


x_train.shape

from pathlib import Path
# Define path to the data directory
data_dir = Path('../input/chest-xray-pneumonia/chest_xray/chest_xray')

# Path to train directory (Fancy pathlib...no more os.path!!)
train_dir = data_dir / 'train'
import cv2
from keras.utils import to_categorical
# Path to validation directory
val_dir = data_dir / 'val'

# Path to test directory
test_dir = data_dir / 'test'

#Get the path to the sub-directories
normal_cases_dir = test_dir / 'NORMAL'
pneumonia_cases_dir = test_dir / 'PNEUMONIA'

# Get the list of all the images
normal_cases = normal_cases_dir.glob('*.jpeg')
pneumonia_cases = pneumonia_cases_dir.glob('*.jpeg')

# List that are going to contain validation images data and the corresponding labels
valid_data = []
valid_labels = []
import numpy as np



# Normal cases
for img in normal_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (150,150))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (150,150)).astype(np.float32)/255.0
    label = to_categorical(0, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
                      
# Pneumonia cases        
for img in pneumonia_cases:
    img = cv2.imread(str(img))
    img = cv2.resize(img, (150,150))
    if img.shape[2] ==1:
        img = np.dstack([img, img, img])
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)/255.0
    label = to_categorical(1, num_classes=2)
    valid_data.append(img)
    valid_labels.append(label)
    
# Convert the list into numpy arrays
x_test = np.array(valid_data)
y_test = np.array(valid_labels)



y_test.shape

In [ ]:
in_shape = Input(shape=(150,150,3))
model = cnn(weights = 'imagenet',input_shape=(150,150,3), include_top=False)

model.summary()

Now we will freeze all layers of our pretrained model

In [ ]:
for layer in model.layers:
      layer.trainable = False

In [ ]:

last_output = model.output

In [ ]:


# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)               
# Add a final sigmoid layer for classification
x = layers.Dense(2, activation='softmax')(x)           

model = Model(model.input, x) 

model.summary()

model=cnn(weights=None, include_top=True,classes=2,input_shape=(150,150,3))
model.summary()

In [ ]:
#sgd=SGD(lr=0.01,momentum=0.9,nesterov=True)
#model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

model.compile(optimizer = Adam(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [ ]:
history = model.fit_generator(
            train_generator,
            validation_data = val_generator,
            steps_per_epoch = nb_train_samples // batch_size,
            epochs = 5,
            validation_steps = nb_val_samples // batch_size,
            verbose = 1)



history=model.fit(x_train,y_train, validation_data=(x_test,y_test), epochs=1, batch_size=10)


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:

# evaluate the model
scores = model.evaluate_generator(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


In [ ]:
model.save("/kaggle/working/mobilenetmodel.h5")

score = model.evaluate(x_test, y_test, verbose=0)
print(score[1]*100)